# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [51]:
import torch
import torch.nn as nn

In [52]:
print(torch.backends.mps.is_available())

True


In [53]:
if torch.backends.mps.is_available():
    device = torch.device("mps") # Apple GPU
    print("Using MPS device")
else:
    device = torch.device("cpu") # Defaults to CPU
    print("MPS device not found, using CPU")

# Example: Move a tensor or model to the MPS device
x = torch.ones(5, device=device)
class TwoBranch(nn.Module):
    def __init__(self):
        super().__init__()
        model = nn.LSTM(input_size=3, hidden_size=5, batch_first=False)
        self.left = model
        self.right = model
        self.combine = nn.Linear(10, 10)

    def forward(self, x_left, x_right):
        _, (l_h, _) = self.left(x_left)
        _, (r_h, _) = self.right(x_right)

        l = l_h[-1]  # last layer, shape: (batch, hidden_size)
        r = r_h[-1]

        cat = torch.cat([l, r], dim=-1)  # shape: (batch, 10)
        return self.combine(cat)          # shape: (batch, 10) -> Linear(10,1) -> (batch,1)
class BigModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.branch = TwoBranch()
        self.rest = nn.Sequential(
            nn.Linear(10, 8),
            nn.ReLU(),
            nn.Linear(8, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
        )

    def forward(self, x_left, x_right):
        x = self.branch(x_left, x_right)
        return self.rest(x)
model2=BigModel()
model2.to(device)



Using MPS device


BigModel(
  (branch): TwoBranch(
    (left): LSTM(3, 5)
    (right): LSTM(3, 5)
    (combine): Linear(in_features=10, out_features=10, bias=True)
  )
  (rest): Sequential(
    (0): Linear(in_features=10, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [54]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)


In [55]:
batch = 32
seq_len = 64

x_left  = torch.randn(seq_len, batch, 3, device=device)
x_right = torch.randn(seq_len, batch, 3, device=device)

# labels depend on problem:
y = torch.randn(batch, 1, device=device)  # regression

In [ ]:
import pandas as pd
pd.read_json()

In [56]:
for epoch in range(1000):
    optimizer.zero_grad()

    output = model2(x_left, x_right)
    loss   = criterion(output, y)

    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(epoch, loss.item())

0 0.7658539414405823
100 0.6640321612358093
200 0.415704607963562
300 0.04017205163836479
400 0.00202933419495821
500 0.000345403648680076
600 5.408459037425928e-05
700 6.412239144992782e-06
800 5.684200914402027e-07
900 3.7777549977136005e-08
